# Cascade Classifier
- 트리 기반 부스트된 거절 캐스케이드 개념을 기초로 하며 얼굴 인식을 목적으로 했다가 이후 그 목적을 일반화해서 얼굴 말고도 대부분의 물체(강체) 인식이 가능함

In [2]:
import cv2
import numpy as np

face_cascade = cv2.CascadeClassifier('./Cascades/haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('./Cascades/haarcascade_eye.xml')

cap = cv2.VideoCapture(0) # 카메라 캡쳐 활성화
while cap.isOpened():
    ret, img = cap.read()
    if ret:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors = 5, minSize = (80, 80))
        for (x, y, w, h) in faces:
            cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
            roi = gray[y:y+h, x:x+w]
            
            eyes = eye_cascade.detectMultiScale(roi)
            for i, (ex, ey, ew, eh) in enumerate(eyes):
                if i >= 2:
                    break
                cv2.rectangle(img[y:y+h, x:x+w], (ex, ey), (ex+ew, ey+eh), (255, 0, 0), 2)
            
        cv2.imshow('face detect', img)
    else:
        break
    if cv2.waitKey(5) == 27:
        break
cv2.destroyAllWindows()
    

# LBPH 얼굴 인식 (Local Binary Pattern Histogram)
- 인식한 얼굴이 누구인지를 알아낼 수 있음.
- 공간적 관계를 유지하면서 지역적 특징을 추출

In [2]:
import cv2
import numpy as np
import os
base_dir = './IMG/train/'
target_cnt = 10 # 수집할 사진 수
cnt = 0 # 사진 촬영 수

face_classifier = cv2.CascadeClassifier('./Cascades/haarcascade_frontalface_default.xml')

name = input("Insert User Name(Only Alphabet): ")
id = input("Insert User Id(Non-Duplicate number): ")
dir = os.path.join(base_dir, name+'_'+id)

if not os.path.exists(dir):
    os.mkdir(dir)

cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    if ret:
        img = frame.copy()
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)
        if len(faces) == 1:
            (x, y, w, h) = faces[0]
            cv2.rectangle(frame, (x,y), (x+w, y+h), (0, 255, 0), 1)
            face = gray[y:y+h, x:x+w]
            face = cv2.resize(face, (200, 200))
            file_name_path = os.path.join(dir, str(cnt) + '.jpg')
            cv2.imwrite(file_name_path, face) # 수집한 사진을 설정한 경로에 저장
            cv2.putText(frame, str(cnt), (x, y), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
            cnt+=1
        else:
            if len(faces) == 0:
                msg = "no face"
            elif len(faces) > 1:
                msg = "too many face"
            cv2.putText(frame, msg, (10, 50), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 0, 255))
        
        cv2.imshow('face record', frame)
        if cv2.waitKey(1) == 27 or cnt == target_cnt:
            break

cap.release()
cv2.destroyAllWindows()
print("Collecting Samples Completed.")

In [1]:
import cv2
import numpy as np
import os
import glob

base_dir = './IMG/train/'
train_data, train_labels = [], []
recognizer = cv2.face.LBPHFaceRecognizer_create()

dirs = [d for d in glob.glob(base_dir+'/*') if os.path.isdir(d)]

print('Collecting train data set...')

for dir in dirs:
    id = dir.split('_')[1]
    files = glob.glob(dir+'/*.jpg')
    print('\t path:%s, %dfiles'%(dir, len(files)))
    
    for file in files:
        img = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
        train_data.append(np.asarray(img, dtype=np.uint8))
        train_labels.append(int(id))
        
train_data = np.asarray(train_data)
train_labels = np.int32(train_labels)

print('Starting LBP Model training...')
model = recognizer
model.train(train_data, train_labels)
model.write('./IMG/train/all_face.xml')
print("Model trained successfully!")

AttributeError: module 'cv2' has no attribute 'face'

In [ ]:
# 훈련한 LBP 얼굴 인식기로 얼굴 인식은 PASS...